## Projeto - Diferenças entre GridSearchCV, RandonSearchCV e Baysian Search - Victor Tintel

In [2]:
# A primeira coisa que temos que fazer é importar os pacotes que iremos utilizar.

import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore") 


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 50)

## Importação dos dados, Analise Exploratória e Tratamento de Dados

In [4]:
# Comando utilizado para carregar o arquivo e armazena-lo como um DataFrame do Pandas

df_dados = pd.read_excel("dados.xlsx")

In [6]:
# Comando utilizado para verificar a quantidade de linhas e colunas do arquivo

df_dados.shape

(2000, 11)

In [8]:
#Comando utilizado para verificar informações sobre os dados(Tipo de variáveis, Variáveis, Quantidade de registros, etc)

df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   UF                          2000 non-null   object 
 1   IDADE                       2000 non-null   int64  
 2   ESCOLARIDADE                2000 non-null   object 
 3   ESTADO_CIVIL                2000 non-null   object 
 4   QT_FILHOS                   2000 non-null   int64  
 5   VL_IMOVEIS                  2000 non-null   int64  
 6   TEMPO_ULTIMO_EMPREGO_MESES  2000 non-null   int64  
 7   ULTIMO_SALARIO              2000 non-null   int64  
 8   VALOR_TABELA_CARROS         2000 non-null   int64  
 9   SCORE_CREDITO               2000 non-null   float64
 10  EMPRESTIMO                  2000 non-null   object 
dtypes: float64(1), int64(6), object(4)
memory usage: 172.0+ KB


In [10]:
# Comando utilizado para avaliar se alguma variável possui valor nulo ou chamado de valores missing ou NAN (Not Available)

df_dados.isnull().sum()

UF                            0
IDADE                         0
ESCOLARIDADE                  0
ESTADO_CIVIL                  0
QT_FILHOS                     0
VL_IMOVEIS                    0
TEMPO_ULTIMO_EMPREGO_MESES    0
ULTIMO_SALARIO                0
VALOR_TABELA_CARROS           0
SCORE_CREDITO                 0
EMPRESTIMO                    0
dtype: int64

## Pré Processamento dos Dados

In [12]:
# Cria o encoder
lb = LabelEncoder()

# Aplica o encoder nas variáveis que estão com string
df_dados['ESTADO_CIVIL'] = lb.fit_transform(df_dados['ESTADO_CIVIL'])
df_dados['ESCOLARIDADE'] = lb.fit_transform(df_dados['ESCOLARIDADE'])
df_dados['UF'] = lb.fit_transform(df_dados['UF'])

In [14]:
df_dados.head()

,UF,IDADE,ESCOLARIDADE,ESTADO_CIVIL,QT_FILHOS,VL_IMOVEIS,TEMPO_ULTIMO_EMPREGO_MESES,ULTIMO_SALARIO,VALOR_TABELA_CARROS,SCORE_CREDITO,EMPRESTIMO
0,4,19,2,2,0,0,8,1800,0,56.000000,SIM
1,0,23,1,2,1,0,9,4800,50000,18.000000,NAO
2,3,25,0,0,0,220000,18,2200,30000,45.000000,SIM
3,1,27,2,0,1,0,22,3900,0,28.666667,NAO
4,4,30,0,1,0,0,14,3100,40000,39.666667,SIM


In [16]:
# Separando a variavel alvo

target = df_dados.iloc[:,10]

In [18]:
target.head()

0    SIM
1    NAO
2    SIM
3    NAO
4    SIM
Name: EMPRESTIMO, dtype: object

In [20]:
# Separando as variaveis preditoras

preditoras = df_dados.copy() # Fazendo uma cópia do dataframe

del preditoras['EMPRESTIMO'] # Excluindo a variavel target, pois já separamos ela na etapa anterior

preditoras.head() # Visualizando as variaveis preditoras

,UF,IDADE,ESCOLARIDADE,ESTADO_CIVIL,QT_FILHOS,VL_IMOVEIS,TEMPO_ULTIMO_EMPREGO_MESES,ULTIMO_SALARIO,VALOR_TABELA_CARROS,SCORE_CREDITO
0,4,19,2,2,0,0,8,1800,0,56.000000
1,0,23,1,2,1,0,9,4800,50000,18.000000
2,3,25,0,0,0,220000,18,2200,30000,45.000000
3,1,27,2,0,1,0,22,3900,0,28.666667
4,4,30,0,1,0,0,14,3100,40000,39.666667


In [22]:
# Divisão em Dados de Treino e Teste.

X_treino, X_teste, y_treino, y_teste = train_test_split(preditoras, target, test_size = 0.3, random_state = 40)

In [26]:
# Vou aplicar a normalização em treino e teste
# Padronização

sc = MinMaxScaler()
X_treino_normalizados = sc.fit_transform(X_treino)
X_teste_normalizados = sc.transform(X_teste)

## Usando o GridSearchCV

In [52]:
# Construtor do Modelo

randomForest = RandomForestClassifier()

In [48]:
# Valores para o grid de hiperparametros

n_estimators = [100, 150, 300]
max_depth = [10, 20]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
min_samples_split = [1, 2]
min_samples_leaf = [1, 2]
bootstrap = [True, False]  # Note que são booleanos, não strings

- Ele vai fazer a combinação de probabilidade com TODOS os parâmetros... n_estimators 100 vai fazer com Max_depth 10, depois com 20, depois com criterion gini, depois entropy... vai passar por todas as combinações possíveis.

In [50]:
# Grid de hiperparâmetros

grid_parametros = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'criterion': criterion,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [56]:
# Crie o GridSearchCV
randomForest = GridSearchCV(
    estimator=randomForest, 
    param_grid=grid_parametros, 
    cv=3, 
    n_jobs=8
)

In [58]:
# Treinando os modelos

inicio = time.time()
randomForest.fit(X_treino_normalizados, y_treino)
fim = time.time()

In [60]:
# Obtendo e visualizando os parametros treinados
treinos_rf = pd.DataFrame(randomForest.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_rf.shape[0])

Acurácia em Treinamento: 77.71%

Hiperparâmetros Ideais: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

Tempo de Treinamento do Modelo:  13.14

Numero de treinamentos realizados:  192


## Usando o RandomizedSearchCV com 50 COMBINAÇÕES

In [62]:
# Construtor do Modelo
randomForest_1 = RandomForestClassifier()

In [64]:
# Valores para o grid de hiperparametros

n_estimators = [100, 150, 300]
max_depth = [10, 20]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
min_samples_split = [1, 2]
min_samples_leaf = [1, 2]
bootstrap = [True, False]  # Note que são booleanos, não strings

In [68]:
# Numero de iterações do RandonSearch
# Aqui no randomsearch podemos dizer quantas combinações queremos fazer 

iteracoes = 50

In [70]:
# Grid de hiperparâmetros

grid_parametros = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'criterion': criterion,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [74]:
# Crie o GridSearchCV

randomForest = RandomizedSearchCV(
    estimator=randomForest_1, 
    param_distributions=grid_parametros, 
    cv=3, 
    n_jobs=8,
    n_iter = iteracoes
)

In [76]:
# Treinando os modelos
inicio = time.time()
randomForest.fit(X_treino_normalizados, y_treino)
fim = time.time()

In [78]:
# Obtendo e visualizando os parametros treinados
treinos_rf = pd.DataFrame(randomForest.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_rf.shape[0])

Acurácia em Treinamento: 78.28%

Hiperparâmetros Ideais: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini', 'bootstrap': True}

Tempo de Treinamento do Modelo:  4.34

Numero de treinamentos realizados:  50


## Usando o RandomizedSearchCV com TODAS AS COMBINAÇÕES

In [90]:
# Construtor do Modelo
randomForest_2 = RandomForestClassifier()

# Valores para o grid de hiperparametros

n_estimators = [100, 150, 300]
max_depth = [10, 20]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]
min_samples_split = [1, 2]
min_samples_leaf = [1, 2]
bootstrap = [True, False]  # Note que são booleanos, não strings

# Grid de hiperparâmetros

grid_parametros = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'criterion': criterion,
    'max_features': max_features,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

# Crie o GridSearchCV

randomForest = RandomizedSearchCV(
    estimator=randomForest_1, 
    param_distributions=grid_parametros, 
    cv=3, 
    n_jobs=8
)

# Treinando os modelos
inicio = time.time()
randomForest.fit(X_treino_normalizados, y_treino)
fim = time.time()

# Obtendo e visualizando os parametros treinados
treinos_rf = pd.DataFrame(randomForest.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {randomForest.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {randomForest.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_rf.shape[0])

Acurácia em Treinamento: 77.43%

Hiperparâmetros Ideais: {'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'entropy', 'bootstrap': False}

Tempo de Treinamento do Modelo:  0.83

Numero de treinamentos realizados:  10


## Utilizando o Baysian Search

In [98]:
# Instalação do Baysian_opt
# !pip install bayesian-optimization

In [96]:
!pip install bayesian-optimization

In [104]:
!pip install scikit-optimize

In [114]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from bayes_opt import BayesianOptimization

In [118]:
# Defina a função de caixa preta para otimizar. 

def black_box_function(C): 
    # C: hiperparâmetro SVC para otimizar. 
    model = SVC(C = C) 
    model.fit(X_treino_normalizados, y_treino) 
    y_score = model.decision_function(X_teste_normalizados) 
    f = roc_auc_score(y_teste, y_score) 
    return f

In [122]:
# Defina o intervalo de C para otimizar. 
# bayes_opt requer que seja um dicionário. 
pbounds = {"C": [0.1, 10]}

# Cria o otimizados e otimiza a função da caixa preta
optimizer = BayesianOptimization(f = black_box_function,
                                 pbounds = pbounds, verbose = 2,
                                 random_state = 4)
optimizer.maximize(init_points = 5, n_iter = 20)
print("Melhor Resultado: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.8789    | 9.674     |
| 2         | 0.878     | 5.518     |
| 3         | 0.8772    | 9.73      |
| 4         | 0.8779    | 7.177     |
| 5         | 0.8802    | 7.008     |
| 6         | 0.8802    | 9.75      |
| 7         | 0.8307    | 0.94      |
| 8         | 0.8805    | 9.911     |
| 9         | 0.8772    | 7.078     |
| 10        | 0.8809    | 7.177     |
| 11        | 0.8797    | 7.175     |
| 12        | 0.8784    | 9.913     |
| 13        | 0.8785    | 9.748     |
| 14        | 0.8764    | 9.751     |
| 15        | 0.8791    | 7.009     |
| 16        | 0.8749    | 9.672     |
| 17        | 0.8774    | 9.675     |
| 18        | 0.8771    | 9.91      |
| 19        | 0.8787    | 9.746     |
| 20        | 0.8797    | 7.01      |
| 21        | 0.8792    | 7.006     |
| 22        | 0.8786    | 7.011     |
| 23        | 0.8766    | 9.745     |
| 24        | 0.8784    | 7.005     |
| 25        